In [1]:
import os
print("Current working directory:", os.getcwd())

Current working directory: /Users/nikhilw/Documents/Devel/PyProj_SQLAgent/simple_db_QnA/App


In [17]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///../data/Sample_DB.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT name FROM sqlite_master LIMIT 10;")
db.run("SELECT * FROM Drugs LIMIT 10;")

sqlite
['Cost_Of_Care', 'Diseases', 'Diseases_By_Channel', 'Drugs', 'Member_Costs', 'Members', 'Pharmacies']


"[('Curr Rank', 'Prev Rank', 'Drug Name', 'Category', 'Utilizers', 'Claims', 'Plan Cost', '% of Total Plan Cost', 'Plan Cost PMPM', 'Plan Cost/Claim*', 'Member Cost/Claim', 'Member Cost Share', 'Prev Utilizers', 'Prev Claims', 'Prev Plan Cost', 'Claims PMPM Trend*', 'Plan Cost/Claim Trend*', 'Plan Cost PMPM Trend', 'Plan Cost Trend', 'Srx'), (1, 18, 'MOUNJARO', 'Diabetes', 7812, 54218, 51838552.35, 0.062, 9.16, 900.13, 33.83, 0.034, 5216, 7080, 6296048.11, 6.073, 0.042, 6.371, 7.234, 'N'), (2, 1, 'HUMIRA', 'Inflammatory Disorder', 899, 7057, 50498194.86, 0.06, 8.92, 7148.67, 927.4, 0.115, 920, 7150, 45587843.11, -0.116, 0.121, -0.008, 0.108, 'Y'), (3, 3, 'STELARA', 'Inflammatory Disorder', 276, 1188, 23055438.25, 0.027, 4.07, 10829.23, 2145.43, 0.1, 251, 1118, 21234652.16, -0.068, 0.043, -0.028, 0.086, 'Y'), (4, 4, 'TRULICITY', 'Diabetes', 3533, 19569, 20053132.6, 0.024, 3.54, 808.72, 41.56, 0.039, 3859, 19037, 20644961.04, -0.163, 0.038, -0.13, -0.029, 'N'), (5, 5, 'BIKTARVY', 'HIV In

In [18]:
db.get_usable_table_names()

['Cost_Of_Care',
 'Diseases',
 'Diseases_By_Channel',
 'Drugs',
 'Member_Costs',
 'Members',
 'Pharmacies']

In [ ]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
	os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

try:
	response = llm.invoke("What is the capital of India?")
	print("Response:", response)
except Exception as e:
	print("Error connecting to OPENAI API:", e)
	print("Please check your internet connection, firewall settings, and API key.")
print("llm:", llm)


In [ ]:
from typing_extensions import TypedDict


class State(TypedDict):
    question: str # User's question to be answered.
    query: str # SQL query generated from the question.
    result: str # Result of the query execution in json.
    answer: str  # Answer to the question based on the query result.
    columns: list[str]  # List of column names in the result.
    followup: list[str] # List of followup questions 